<a href="https://colab.research.google.com/github/agnesedaff/Implicit_obj_completion/blob/main/EXPLICIT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install transformers

In [ ]:
!python -m pip install spacy
!python -m spacy download it_core_news_lg

2024-01-17 15:25:04.480789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 15:25:04.480861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 15:25:04.482191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 15:25:04.490437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-17 15:25:06.572042: W tensorflow/compiler/tf2

In [ ]:
import spacy
nlp = spacy.load("it_core_news_lg")

In [ ]:
# Prova del modello sull'EXPLICIT dataset con 1 token [MASK].

from transformers import pipeline

# Initialize the fill-mask pipeline with the Italian model
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased', tokenizer = 'bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

**Primo step:** otteniamo i risultati del modello

In [ ]:
documento = open('/content/drive/MyDrive/explicit_obj_dataset.txt','r')
sentences = documento.readlines()

In [ ]:
output = []
scores = []

for sentence in sentences:
    noun_predictions = []
    results = unmasker(sentence, top_k=10) #usiamo l'unmasker per ottenere i primi 10 risultati
    for prediction in results: #per ogni risultato, applichiamo il pos tagging di SpaCy in modo da isolare solo i nomi
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction) #aggiungiamo i nomi candidati alla lista noun_predictions
    if noun_predictions:
        noun_predictions.sort(key=lambda x: x["score"], reverse=True) #ordiniamo i candidati in base al prediction score
        output.append(noun_predictions[0]["token_str"]) # isoliamo il candidato con lo score più alto e stampiamo il punteggio e la frase completa
        scores.append(round(noun_predictions[0]["score"], 2))
    else:
        output.append("unknown")
        scores.append("unknown")
print(output)
print(scores)

['musica', 'canto', 'suono', 'voce', 'carriera', 'suono', 'ragazzi', 'concerto', 'voce', 'voce', 'segno', 'libri', 'canto', 'audio', 'nome', 'voce', 'colori', 'rapporto', 'nome', 'processi', 'studi', 'festa', 'unknown', 'periodo', 'autonomia', 'arrivo', 'morte', 'sorella', 'tempo', 'nemico', 'uscita', 'unknown', 'inizio', 'notte', 'soluzione', 'visione', 'tempio', 'definizione', 'Dio', 'inizio', 'note', 'città', 'olio', 'luce', 'unknown', 'cibo', 'unknown', 'cibo', 'giro', 'carne', 'giorno', 'testa', 'mezzo', 'cielo', 'cibo', 'unknown', 'mondo', 'unknown', 'vino', 'vino', 'brani', '##nti', 'unknown', 'brani', 'musica', 'grazie', 'chitarra', 'unknown', 'pianoforte', 'canto', 'unknown', 'unknown', 'opera', 'aria', 'poesia', 'note', 'giudice', 'parti', 'testi', 'canto', 'nemici', 'polizia', 'finestra', 'città', 'personaggi', 'nemici', 'uomo', 'computer', 'principi', 'padre', 'giorno', 'comandante', 'moglie', 'fiori', 'agenti', 'persona', 'navi', 'ragazze', 'campi', 'sorella', 'guerra', 'b

**Secondo step:** calcoliamo la similarity tra la parola originale e il risultato del modello

In [ ]:
# Per calcolare la similarity tra le parole predicted e l'output del modello:
# Creiamo un dizionario che associa le coppie dell'una e dell'altra lista
# Per evitare sovrapposizioni, aggiungiamo una numerazione alle parole ripetute

predicted = ['musica', 'canto', 'versi', 'musica', 'voce', 'suono', 'suoni', 'concerto', 'voce', 'voce', 'grido', 'canti', 'canto', 'audio', 'suono', 'voce', 'rumori', 'concerto', 'respiro', 'concerti', 'chiarimenti', 'sfida', 'Pasqua', 'presentazione', 'approvazione', 'esame', 'visita', 'partenza', 'turno', 'aiuto', 'arrivo', 'risposta', 'apertura', 'morte', 'asciugatura', 'rinascita', 'ritorno', 'conclusione', 'conferme', 'uscita', 'birra', 'acqua', 'estratto', 'acqua', 'latte', 'alcool', 'acqua', 'vino', 'bicchiere', 'alcolici', 'drink', 'cicuta', 'caffè', 'the', 'mate', 'sangue', 'tè', 'bevande', 'cappuccino', 'whisky', 'brani', 'ninnananne', 'canzoni', 'testi', 'serenata', 'inni', 'filastrocca', 'cori', 'ritornello', 'mantra', 'lodi', 'Money', 'musiche', 'aria', 'canzone', 'composizioni', 'motivo', 'strofe', 'pezzi', 'salmo', 'carabinieri', 'polizia', 'polizia', 'polizia', 'medici', 'vigili', 'amico', 'amico', 'vigili', 'medico', 'dottore', 'madre', 'madre', 'vigili', 'vigili del fuoco', 'persona', 'persone', 'persone', 'pompieri', 'figlia', 'battaglia', 'guerra', 'battaglie', 'guerra', 'battaglia', 'guerra', 'guerra', 'guerra', 'guerra', 'guerra', 'battaglia', 'battaglia', 'battaglia', 'battaglia', 'battaglia', 'lotta', 'lotta', 'lotta', 'lotta', 'lotte', 'telegiornale', 'programmi', 'telegiornale', 'trasmissione', 'Il ciclotrone', 'show', 'Media-mente', 'Affari Tuoi', 'programma', 'trasmissione', 'diretta', 'programma', 'dibattito', 'telegiornale', 'edizione', 'trasmissione', 'quiz', 'serate', 'edizione', 'intervista', 'carburante', 'petrolio', 'energia', 'CO2', 'risorse', 'energia', 'copertoni', 'risorse', 'petrolio', 'volt', 'ossigeno', 'batterie', 'ricchezza', 'kilocalorie', 'inchiostro', 'riserve', 'farmaci', 'punta', 'benzina', 'elettricità', 'maratona', 'maratona', 'gare', 'gare', 'gara', 'Tour', 'Giro', 'Giro', 'Gran Premio', 'palio', 'campionato', 'campionato', 'Mondiale', 'Mondiale', 'granfondo', 'staffetta', 'tappa', 'prova', 'finali', 'rally', 'ragù', 'peperoni', 'pollo', 'pizza', 'lenticchie', 'aragosta', 'spaghetti', 'piselli', 'molluschi', 'cibo', 'dolci', 'pasta', 'salmone', 'cibi', 'pasti', 'piatti', 'pranzo', 'pesce', 'carne', 'ricette', 'Totò', 'film', 'concerto', 'attrice', 'concerto', 'concerto', 'orchestra', 'Orchestra', 'banda', 'coro', 'esecuzioni', 'opera', 'archi', 'film', 'film', 'episodio', 'cortometraggi', 'commedia', 'spettacoli', 'documentario', 'piantina', 'albero', 'autoritratto', 'limoni', 'oggetti', 'corpo', 'grafici', 'Palazzo', 'tavole', 'fumetti', 'copertine', 'Daredevil', 'mappa', 'cerchi', 'contorno', 'rettangolo', 'linea', 'figure', 'paesaggio', 'vignette', 'sigarette', 'canne', 'narghilè', 'marijuana', 'sigaretta', 'spinello', 'canne', 'sigarette', 'spinello', 'erba', 'marijuana', 'erba', 'sigaretta', 'spinello', 'marijuana', 'sigari', 'pipa', 'pipa', 'pipa', 'tabacco', 'stagione', 'partita', 'mani', 'partite', 'partita', 'fase', 'sfida', 'finale', 'gara', 'gara', 'minuti', 'campionato', 'stagione', 'torneo', 'match', 'tempo', 'derby', 'semifinali', 'Coppa', 'turno', 'percentuale', 'lire', 'stipendio', 'soldi', 'soldi', 'euro', 'denaro', 'somme', 'denaro', 'soldi', 'dollaro', 'milioni', 'soldi', 'soldi', 'soldi', 'denaro', 'soldi', 'soldo', 'soldi', 'miseria', 'veicoli', 'auto', 'automodello', 'auto', 'autobus', 'moto', '4X4', 'Fiat 520', 'catorcio', 'macchina', 'camion', 'carro', 'carro', 'motorini', 'autobus', 'autobus', 'scooter', 'taxi', 'mezzo', 'automobile', 'storie', 'pagina', 'racconti', 'post', 'articolo', 'testo', 'numero', 'Bibbia', 'quotidiani', 'messaggio', 'documenti', 'haiku', 'opere', 'giornale', 'sceneggiatura', 'cartelloni', 'libretto', 'libro', 'saggio', 'romanzo', 'carne', 'cibo', 'insalata', 'riso', 'pesce', 'verdura', 'tartufo', 'hamburger', 'panino', 'pizza', 'spaghetti', 'gelato', 'torta', 'pesce', 'frutta', 'mela', 'piatto', 'pasta', 'uova', 'minestra', 'pietanza', 'birra', 'libro', 'prodotto', 'tortellini', 'copie', 'caffè', 'birre', 'dipinto', 'birre', 'bicchiere', 'pizza', 'cocktail', 'hamburger', 'pasta', 'succo', 'torta', 'pizza', 'pranzo', 'piatti', 'pena', 'errori', 'conseguenze', 'colpe', 'debito', 'conseguenze', 'conseguenze', 'conseguenze', 'prezzo', 'prezzo', 'pegno', 'scelte', 'incoerenza', 'pegno', 'fio', 'errore', 'errori', 'ribellione', 'impegno', 'errore', 'mondiali', 'gare', 'battaglia', 'guerra', 'guerra', 'dibattiti', 'guerra', 'partita', 'partita', 'partita', 'partita', 'battaglia', 'battaglie', 'battaglia', 'guerra', 'match', 'match', 'gara', 'gara', 'gara', 'divinità', 'Dio', 'Signore', 'Padre', 'Dio', 'Galileo', 'Indra', 'Dio', 'divinità', 'Dio', 'Sole', 'studente', 'Dirigenti scolastici', 'uffici', 'Direzioni Regionali', 'parroco', 'lettore', 'amici', 'insegnante', 'nipote', 'Mancini', 'portotorresi', 'Napoleone III', 'Gruppo di lavoro', 'sindacati', 'genitori', 'esperti', 'ambientalisti', 'autorità', 'italiani', 'cittadini', 'europei', 'mamme', 'gestori', 'specialisti', 'intellettuali', 'dirigenti', 'governo', 'Chiesa', 'scienziati', 'arrangiamenti', 'brani', 'canti', 'regolazioni', 'brano', 'brani', 'brano', 'accordi', 'brani', 'spettacolo', 'canzone', 'spettacolo', 'spettacolo', 'scene', 'coreografia', 'canzoni', 'canzone', 'canzoni', 'musica', 'pezzi', 'fumi', 'aria', 'fumo', 'profumo', 'esalazioni', 'profumo', 'ossigeno', 'ossigeno', 'aria', 'vapori', 'odore', 'aria', 'aria', 'smog', 'smog', 'gas', 'gas', 'aromi', 'alito', 'profumo', 'post', 'testi', 'versi', 'articoli', 'poesìa', 'commento', 'libri', 'lettera', 'articoli', 'poesie', 'romanzo', 'storia', 'messaggi', 'post', 'libro', 'pagina', 'codice', 'mail', 'libro', 'opinione', 'colazione', 'piatti', 'pasti', 'pasti', 'cena', 'clienti', 'clienti', 'caffè', 'caffè', 'vino', 'pranzo', 'pranzi', 'pranzo', 'zuppa', 'cibo', 'cibo', 'cibo', 'birre', 'birra', 'birra', 'fisarmonica', 'tamburo', 'flauto', 'tromba', 'piano', 'pezzo', 'musica', 'concerti', 'campanello', 'campanello', 'allarme', 'campana', 'musica', 'clacson', 'canzoni', 'strumenti', 'brano', 'pezzi', 'pezzo', 'jazz', 'pallone', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palla', 'palle', 'palla', 'palla', 'pallone', 'pallone', 'pallone', 'pallone', 'battaglia', 'incontro', 'Liga', 'corsa', 'tappa', 'mondiale', 'Eurofestival', 'tappa', 'campionato', 'elezioni', 'elezioni', 'concorso', 'battaglia', 'concorso', 'gara', 'campionato', 'partita', 'guerra', 'guerra', 'competizione']
word_dict = {}
counter = {}

for predicted_word, output_word in zip(predicted, output):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word

print("word_dict: ", word_dict)
print(len(word_dict))

word_dict:  {'musica': 'musica', 'canto': 'canto', 'versi': 'suoni', 'musica_1': 'musica', 'voce': 'voce', 'suono': 'rumore', 'suoni': 'segnali', 'concerto': 'concerto', 'voce_1': 'amica', 'voce_2': 'voce', 'grido': 'grido', 'canti': 'canti', 'canto_1': 'sorriso', 'audio': 'audio', 'suono_1': 'suono', 'voce_3': 'voce', 'rumori': 'rumori', 'concerto_1': 'pezzo', 'respiro': 'respiro', 'concerti': 'concerti', 'chiarimenti': 'chiarimenti', 'sfida': 'vittoria', 'Pasqua': 'tempo', 'presentazione': 'termine', 'approvazione': 'autorizzazione', 'esame': 'approvazione', 'visita': 'risposta', 'partenza': 'visita', 'turno': 'momento', 'aiuto': 'turno', 'arrivo': 'arrivo', 'risposta': 'risposta', 'apertura': 'apertura', 'morte': 'notte', 'asciugatura': 'guarigione', 'rinascita': 'rinascita', 'ritorno': 'ritorno', 'conclusione': 'conclusione', 'conferme': 'aggiornamenti', 'uscita': 'uscita', 'birra': 'birra', 'acqua': 'acqua', 'estratto': 'acqua', 'acqua_1': 'acqua', 'latte': 'latte', 'alcool': 'alc

In [ ]:
# Calcoliamo la similarity tra le parole del dizionario con SpaCy.
# Per fare questo dobbiamo sistematicamente rimuovere il numero di indice aggiunto prima.

import re

# Funzione per calcolare la similarity tra due parole
def calculate_similarity(word1, word2):
    word1 = re.sub(r"_\d+", "", word1)
    word2 = re.sub(r"_\d+", "", word2)
    word1_token = nlp(word1)
    word2_token = nlp(word2)
    if word2 == "unknown":
        return "unknown"
    if word1_token.has_vector and word2_token.has_vector:
        similarity = word1_token.similarity(word2_token)
        similarity_rounded = round(similarity, 2)
        return similarity_rounded
    else:
        return "unknown"

# Calcola la similarity per ogni coppia nel dizionario

similarity_scores = []
for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores.append(similarity)
print(similarity_scores)

[1.0, 1.0, 0.53, 1.0, 1.0, 0.69, 0.49, 1.0, 0.29, 1.0, 1.0, 1.0, 0.36, 1.0, 1.0, 1.0, 1.0, 0.29, 1.0, 1.0, 1.0, 0.58, 0.05, 0.38, 0.85, 0.52, 0.27, 0.42, 0.3, 0.15, 1.0, 1.0, 1.0, 0.3, 0.48, 1.0, 1.0, 1.0, 0.46, 1.0, 1.0, 1.0, 0.36, 1.0, 1.0, 0.69, 1.0, 0.54, 1.0, 1.0, 0.61, 0.33, 0.77, -0.16, 0.32, 1.0, 0.77, 1.0, 0.69, 0.54, 1.0, 0.08, 1.0, 0.68, 0.32, 0.01, 0.59, 0.33, 0.49, 0.47, 0.14, 0.25, 1.0, 0.14, 0.57, 0.59, 0.31, 0.65, 0.61, 0.44, 1.0, 1.0, 0.8, 1.0, 0.25, 0.69, 0.15, 1.0, 1.0, 1.0, 0.54, 0.82, 0.82, 0.69, 0.47, 1.0, 1.0, 1.0, 0.67, 1.0, 0.69, 0.69, 1.0, 1.0, 0.33, 0.69, 1.0, 0.33, 0.69, 0.13, 0.7, 1.0, 1.0, 1.0, 1.0, 0.7, 1.0, 0.29, 0.7, 1.0, 0.42, 1.0, 0.23, 1.0, 0.1, 0.63, 0.45, 0.25, 1.0, 0.44, 0.07, 1.0, 1.0, 0.42, 1.0, 1.0, 0.35, 0.59, 1.0, 1.0, 0.27, 0.29, 1.0, 0.53, 1.0, 1.0, 0.57, 1.0, 0.59, 0.44, 0.6, 1.0, 1.0, 0.84, 0.32, 0.68, 1.0, 0.24, 0.71, 0.83, 1.0, 0.24, 1.0, 0.21, 0.2, 0.25, 0.4, 0.4, 0.33, 0.47, 0.12, 1.0, 0.08, 0.54, 0.43, 0.54, 1.0, 0.54, 0.32, 1.0, 0.4